# Étape 3 - Analyse des réseaux sociaux

*Durée prévisionnelle : 1h15*

**Présentation slides avant le notebook**

Il est possible de s'intéresser non pas uniquement au comptage de certains éléments, mais au lien que ces éléments ont ensemble.

Dans le cas des des médias sociaux, le plus évident est le lien entre les utilisateurs.

Il peut y avoir différents types de liens :
- le fait de suivre quelqu'un
- le fait de mentionner quelqu'un
- le fait de partager certaines informations (retweet) 
- le fait d'avoir des caractéristiques similires (similarité)

Étudier les réseaux permet de s'intéresser à des formes de proximité / distance entre des entités.

## 1. Construire un réseau égocentrique et visualiser

Pour cela nous allons utiliser la bibliothèque Networkx qui permet de construire et de manipuler des structures de graph : https://networkx.org/

### Construire le réseau de tous les comptes mentionnés par Didier Raoult

In [1]:
import pandas as pd
df = pd.read_parquet("dataset.parquet")

Uniquement les tweets de Didier Raoult

In [2]:
comptes = df[df["from_user_name"]=="raoult_didier"]["mentioned_user_names"].dropna()

In [3]:
list(comptes)

['ihu_marseille', 'raoult_didier', 'ihu_marseille', 'olivierveran', 'aphp']

Bibliothèque pour les réseaux

In [4]:
import networkx as nx

Créer un objet réseau

In [5]:
reseau = nx.Graph()

In [15]:
type(reseau)

networkx.classes.graph.Graph

In [6]:
# ajouter raoult
reseau.add_node("raoult",poids=1)

# pour chaque compte mentionné
for ent in comptes:
    # ajouter le compte
    if reseau.has_node(ent):
        reseau.nodes[ent]["poids"]+=1
    else:
        reseau.add_node(ent,poids=1)
    # ajouter un lien
    reseau.add_edge(ent,"raoult")

In [7]:
reseau.edges

EdgeView([('raoult', 'ihu_marseille'), ('raoult', 'raoult_didier'), ('raoult', 'olivierveran'), ('raoult', 'aphp')])

In [8]:
reseau.degree

DegreeView({'raoult': 4, 'ihu_marseille': 1, 'raoult_didier': 1, 'olivierveran': 1, 'aphp': 1})

### Visualiser un réseau avec la bibliothèque IPysigma

In [9]:
import ipysigma

In [11]:
ipysigma.Sigma(reseau,node_size="poids")

Sigma(nx.Graph with 5 nodes and 4 edges)

> Installer la bibliothèque IPysigma https://github.com/medialab/ipysigma et visualiser le réseau égocentrique de Didier Raoult

### Bonus : exporter en format graphml et visualiser avec le logiciel Gephi

## 2. Réseau complet de reweets

Un retweet est généralement traduit par une approbation.

### construire le réseau de retweet avec comme noeuds les comptes, leur poids étant le nombre de tweets fait, le lien étant qu'un compte a retweeté un autre, non-orienté

> Se concentrer uniquement sur le début de l'épidémie, mars-avril-mai 2020 et 

In [36]:
df.columns

Index(['id', 'created_at', 'from_user_name', 'text', 'lang',
       'from_user_tweetcount', 'from_user_followercount',
       'from_user_friendcount', 'retweeted_id', 'quoted_user_id',
       'mentioned_user_names', 'count'],
      dtype='object')

In [37]:
df["mentioned_user_names"]

0                                 None
1                                 None
3                                 None
5                                 None
7                                cnews
                      ...             
3766096                     mediavenir
3766097                     echabriere
3766098    dupontaignan|thierrylfronde
3766099                          irfmx
3766100                 silvano_trotta
Name: mentioned_user_names, Length: 3764273, dtype: object

In [50]:
reseau_retweet.edges["CesarJeanRony3","ambassadechine"]

{'poids': 1}

In [52]:
# crée un graph
reseau_retweet = nx.Graph()

# pour chaque tweet
for i,j in df_ss.iterrows():
    
    # si c'est un tweet original
    if pd.notnull(j["retweeted_id"]) & pd.notnull(j["mentioned_user_names"]):
        users = j["mentioned_user_names"].split("|")
        username = j["from_user_name"]
        
        # ajouter le compte qui a tweeté
        if reseau_retweet.has_node(username):
            reseau_retweet.nodes[username]["poids"]+=1
        else:
            reseau_retweet.add_node(username,poids=1)
            
        # pour chaque utilisateur mentionné
        for u in users:
            
            # créer ou augmenter le poids du noeuds
            if reseau_retweet.has_node(u):
                reseau_retweet.nodes[u]["poids"]+=1
            else:
                reseau_retweet.add_node(u,poids=1)
                
            # créer ou augmenter le poids du lien
            if reseau_retweet.has_edge(u,username):
                reseau_retweet.edges[u,username]["poids"] += 1
            else:
                reseau_retweet.add_edge(u,username,poids=1)

In [55]:
len(reseau_retweet.nodes),len(reseau_retweet.edges)

(224625, 1100515)

In [15]:
df_ss = df[df["created_at"]<"2020-04-01"]

# Création du réseau de retweet

network_retweets = nx.Graph()

##  Pour information: 
# utilisateur = 1 compte
# utilisateur["weight"] = Nombre de tweet de l'utilisateur
# retweet_utilisateur = Personne retweetée (au moins 1 fois)
# edges: Nombre de RT fait par utilisateur du compte retweet_utilisateur

# On fait une boucle pour chaque tweet du corpus
for i, line in df_ss.iterrows():
    
    # Nom du compe ayant tweeté
    utilisateur = str(line["from_user_name"])
    
    # L'utilisateur est-il déjà dans le réseau ?
    if not utilisateur in network_retweets.nodes:
        # S'il n'y est pas, on l'ajoute avec un poids de 1
        network_retweets.add_node(utilisateur, label=utilisateur, weight=1)
    else:
        # S'il l'est, on augmente son poids de 1
        network_retweets.nodes[utilisateur]["weight"] += 1    

    # Si le tweet est un retweet
    if pd.notnull(line["retweeted_user_name"]):

        # on extrait le nom de l'auteur original du tweet 
        retweet_utilisateur = str(line["retweeted_user_name"])
        
        # si cet auteur original n'est pas dans le réseau
        if not retweet_utilisateur in network_retweets.nodes:
            # on ajoute un noeud
            network_retweets.add_node(retweet_utilisateur, label=retweet_utilisateur, weight=0)

        # s'il y a un retweet
        if pd.notnull(retweet_utilisateur):
            
            # si l'arrête entre l'utilisitateur et l'auteur original du tweet n'existe pas
            if not network_retweets.has_edge(utilisateur, retweet_utilisateur):
                # On créé cette arrête
                network_retweets.add_edge(utilisateur, retweet_utilisateur, weight=1)
            else:
                # si elle existe déjà on on augmente son poids de 1
                network_retweets[utilisateur][retweet_utilisateur]["weight"] += 1 



### Calculer le nombre de noeuds, le nombre de lien, la densité du réseau

## 3. Filtrer le réseau et le visualiser

### Calculer combien de composantes (éléments déconnectés) a le réseau ; filtrer pour ne garder que la composante principale

- quelle proportion du total du réseau représente-t-elle ?

In [ ]:
# Création du sous-graphe de la composante principale (sur laquelle on se concentre)
list_subgraph = list(nx.connected_components(network_retweets))
list_subgraph.sort(key=len, reverse=True)
connected_graph = network_retweets.subgraph(list_subgraph[0])

### Visualiser le réseau avec IPysigma

> S'il est trop volumineux, comment le réduire ?

## 4. (Pas traité en cours) Clusteriser le réseau pour identifier les communautés

### Utiliser l'algorithme de Louvain pour clusteriser le réseau

> https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.louvain.louvain_communities.html

### Garder les 10 principales communautés et réunir les autres dans une catégorie "autre"

In [ ]:
import networkx.community as nx_com

# On génénère la clusterisation avec l'algorithme de Louvain
cluster_multiples = nx_comm.louvain_communities(connected_graph, weight='weight', 
                                                resolution=1, threshold=1e-07, seed=123)
# Trier du plus grand au plus petit
cluster_multiples = sorted(cluster_multiples,key=len,reverse=True)

# Liste des clusters
liste_clusters = pd.Series([len(i) for i in sorted(cluster_multiples)])

# Regroupement des clusters après le rang 7
cluster8 = set()
for i in cluster_multiples[8:]:
    cluster8.update(i)

# Liste des clusters
cluster_7 =  cluster_multiples[0:7] + [cluster8]

### Essayer de donner un sens aux communautés identifiées

> Calculer pour chaque communauté les comptes les plus centraux et essayer de comprendre qui ils sont

## 5. Bonus - Prendre en compte une dimension temporelle

Pour le moment, nous avons agrégé sur l'ensemble de la période. Maintenant nous voulons essayer d'injecter une composante temporelle. 

- Calculez le réseau par semaine et essayer de tracer l'évolution de ses caractéristiques (taille, densité, modularité)
- Comment faire pour essayer de représenter l'évolution temporelle d'un réseau ?